In [1]:
import numpy as np
import matplotlib.pyplot as plt

def grad(x,function, h):
    n = len(x)
    fprime = np.zeros(n)
    for i in range(n):
        x_aux = np.copy(x)
        x_aux[i] = x[i]+h
        delante = function(x)
        x_aux[i] = x[i]-2*h
        atras = function(x_aux)
        fprime[i] = (delante-atras)/(2*h)
    return fprime

def hessian(x, function, h):
    n = len(x)
    hessian = np.zeros([n, n])
    x_aux = np.copy(x)
    
    for i in range(n):
        for j in range(n):
            x[i] = x[i]+h 
            x[j] = x[j]+h
            d = function(x)
            x[j] = x[j]-2*h
            a = function(x)
            #regresar al valor de x original
            x = np.copy(x_aux)

            x[i] = x[i]-h 
            x[j] = x[j]+h
            dd = function(x)
            x[j] = x[j]-2*h
            aa = function(x)
            #regresar al original
            x = np.copy(x_aux)
            
            #llenar matriz
            hessian[i][j] = (d-a-dd+aa)/(4*h*h)

    return hessian

def rosenbrock(x):
    n = len(x)
    suma = 0
    for i in range(n-1):
        suma += 100*(x[i+1]-x[i]**2)**2+(1-x[i])**2
    return suma

def wood(x):
    return sum((
        100*(x[0]*x[0] - x[1])**2,
        (x[0]-1)**2,
        (x[2]-1)**2,
        90*(x[2]*x[2] - x[3])**2,
        10.1*((x[1]-1)**2 + (x[3]-1)**2),
        19.8*(x[1]-1)*(x[3]-1),
        ))

def branin(x):
    a = 1.0
    b = 5.1 / (4*np.pi**2)
    c = 5.0 / (np.pi)
    r = 6.0
    s = 10.0
    t = 1.0 / (8*np.pi)
    
    return a*(x[1]-b*x[0]**2+c*x[0]-r)**2+s*(1-t)*np.cos(x[0])+s

In [4]:
def rtr(x_k, max_iter, F):
    #para la primer iteracion
    #constantes del articulo
    n = len(x_k)
    gamma1 = 0.25
    gamma3 = 3.5
    eta1 = 0.0001
    eta2 = 0.99
    delta = 1.0
    eta = 0.1
    gamma_g = min(0.001, 1/(2*np.sqrt(n)))
    x_mas = np.zeros(n)
    
    f_k = rosen(x_k)
    g = rosen_der(x_k)
    norm_grad = np.linalg.norm(g)
    B = rosen_hess(x_k)

    #calcular p_k
    p_k = -1.0*(delta/norm_grad)*g
    x_mas = x_k + p_k
    grad_x_mas = rosen_der(x_mas)

    rho_num = rosen(x_k)-rosen(x_mas)
    rho_dev = -np.dot(g.T, x_mas)-(0.5)*np.dot(np.dot(x_mas.T, B), x_mas)
    rho = rho_num / rho_dev

    if (rho >= eta1):
            x_k = x_mas
    elif (rho < eta1 and np.linalg.norm(grad_x_mas) <= F):
            x_k = x_mas
            F = np.linalg.norm(grad_x_mas)

    for i in range(1, max_iter):
        f_k = rosen(x_k)
        g = rosen_der(x_k)
        norm_grad = np.linalg.norm(g)
        #print(norm_grad)
        B = rosen_hess(x_k)

        #calcular p_k
        p_k = -1.0*(delta/norm_grad)*g

        rho_num = rosen(p_k)-rosen(x_k)
        rho_dev = -np.dot(g.T, p_k)-(0.5)*np.dot(np.dot(p_k.T, B), p_k)
        rho = rho_num / rho_dev

        if (rho < eta1):
            delta *= gamma1
        elif (eta1 <= rho <= eta2):
            delta *= gamma2
        else: 
            delta *= gamma3

        x_mas = x_k + p_k
        grad_x_mas = rosen_der(x_mas)

        rho_num = rosen(x_k)-rosen(x_mas)
        rho_dev = -np.dot(g.T, x_mas)-(0.5)*np.dot(np.dot(x_mas.T, B), x_mas)
        rho = rho_num / rho_dev

        if (rho >= eta1):
            x_k = x_mas
        elif (rho < eta1 and np.linalg.norm(grad_x_mas) <= F):
            x_k = x_mas
            F = np.linalg.norm(grad_x_mas)
    return x_k

## Función Rosenbrock

In [5]:
from scipy.optimize import rosen, rosen_der, rosen_hess
import numpy as np
#Condiciones 
n = 100
x_k = np.ones(n) + np.random.uniform(-2, 2, n)
max_iter = 5000
alpha = 0.02
delta = 2
eta = 0.1
tol = 0.01
h = 0.000001
res = np.ones(n)
mean = np.zeros(30)
F = 10000

for i in range(30):
    print(i)
    x_k = np.ones(n) + np.random.uniform(-2, 2, n)
    x = rtr(x_k, max_iter, F)
    mean[i] = (x[0]-res).mean()

0


C:\Users\zaira\AppData\Local\Temp/ipykernel_10800/990706688.py:46: RuntimeWarning: overflow encountered in double_scalars
  rho = rho_num / rho_dev
C:\Users\zaira\AppData\Local\Temp/ipykernel_10800/990706688.py:46: RuntimeWarning: divide by zero encountered in double_scalars
  rho = rho_num / rho_dev


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [6]:
mean.mean()

-0.03020481094425844

## Función Wood

In [7]:
def rtr(x_k, funcion, h, max_iter, F):
    #para la primer iteracion
    #constantes del articulo
    n = len(x_k)
    gamma1 = 0.25
    gamma2 = 0.2
    gamma3 = 3.5
    eta1 = 0.0001
    eta2 = 0.99
    delta = 1.0
    eta = 0.1
    gamma_g = min(0.001, 1/(2*np.sqrt(n)))
    x_mas = np.zeros(n)
    
    f_k = funcion(x_k)
    g = grad(x_k, funcion, h)
    norm_grad = np.linalg.norm(g)
    B = hessian(x_k, funcion, h)

    #calcular p_k
    p_k = -1.0*(delta/norm_grad)*g
    x_mas = x_k + p_k
    grad_x_mas = grad(x_mas, funcion, h)

    rho_num = funcion(x_k)-funcion(x_mas)
    rho_dev = -np.dot(g.T, x_mas)-(0.5)*np.dot(np.dot(x_mas.T, B), x_mas)
    rho = rho_num / rho_dev

    if (rho >= eta1):
            x_k = x_mas
    elif (rho < eta1 and np.linalg.norm(grad_x_mas) <= F):
            x_k = x_mas
            F = np.linalg.norm(grad_x_mas)

    for i in range(1, max_iter):
        f_k = funcion(x_k)
        g = grad(x_k, funcion, h)
        norm_grad = np.linalg.norm(g)
        #print(norm_grad)
        B = hessian(x_k, funcion, h)

        #calcular p_k
        p_k = -1.0*(delta/norm_grad)*g

        rho_num = funcion(p_k)-funcion(x_k)
        rho_dev = -np.dot(g.T, p_k)-(0.5)*np.dot(np.dot(p_k.T, B), p_k)
        rho = rho_num / rho_dev

        if (rho < eta1):
            delta *= gamma1
        elif (eta1 <= rho <= eta2):
            delta *= gamma2
        else: 
            delta *= gamma3

        x_mas = x_k + p_k
        grad_x_mas = grad(x_mas, funcion, h)

        rho_num = funcion(x_k)-funcion(x_mas)
        rho_dev = -np.dot(g.T, x_mas)-(0.5)*np.dot(np.dot(x_mas.T, B), x_mas)
        rho = rho_num / rho_dev
        #print(rho, np.linalg.norm(grad_x_mas), F, eta1)
        if (rho >= eta1):
            x_k = x_mas
        elif (rho < eta1 and np.linalg.norm(grad_x_mas) <= F):
            x_k = x_mas
            F = np.linalg.norm(grad_x_mas)
    return x_k

In [8]:
n = 4
x_k = np.ones(n) + np.random.uniform(-2, 2, n)
max_iter = 100
alpha = 0.02
delta = 2
eta = 0.1
tol = 0.01
h = 0.000001
F = 10000
res = np.ones(n)
mean = np.zeros(30)

for i in range(30):
    print(i)
    x_k = np.ones(n) + np.random.uniform(-2, 2, n)
    x = rtr(x_k, wood, h, max_iter, F)
    mean[i] = (x[0]-res).mean()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [9]:
mean.mean()

-0.6344308250247256

## Función Branin

In [24]:
n = 2
x_k = np.zeros(n)
x_k[0] = np.pi + np.random.uniform(-2, 2)
x_k[1] = 2.275 + np.random.uniform(-2, 2)

max_iter = 1000
alpha = 0.02
delta = 2
eta = 0.1
tol = 0.01
h = 0.000001
res = np.zeros(n)
res[0] = np.pi
res[1] = 2.275
F = 10000

mean = np.zeros(30)

for i in range(30):
    print(i)
    x_k = np.zeros(n)
    x_k[0] = np.pi + np.random.uniform(-2, 2)
    x_k[1] = 2.275 + np.random.uniform(-2, 2)
    
    x = rtr(x_k, branin, h, max_iter, F)
    mean[i] = (x[0]-res).mean()

0
1


C:\Users\zaira\AppData\Local\Temp/ipykernel_10800/1669480683.py:47: RuntimeWarning: overflow encountered in double_scalars
  rho = rho_num / rho_dev
C:\Users\zaira\AppData\Local\Temp/ipykernel_10800/1669480683.py:47: RuntimeWarning: divide by zero encountered in double_scalars
  rho = rho_num / rho_dev
C:\Users\zaira\AppData\Local\Temp/ipykernel_10800/1669480683.py:43: RuntimeWarning: divide by zero encountered in double_scalars
  p_k = -1.0*(delta/norm_grad)*g
C:\Users\zaira\AppData\Local\Temp/ipykernel_10800/1669480683.py:43: RuntimeWarning: invalid value encountered in multiply
  p_k = -1.0*(delta/norm_grad)*g


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [25]:
mean.mean()

-12706026290.664791